# Preguntas y Respuestas con RAG Langchain

Usando Chroma DB and LangChain para contestar preguntas de un texto, con una base de datos local. Guardamos los embaddings, y los usamos después.

In [14]:
!pip install chromadb langchain langchain-openai

In [1]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings

In [2]:
import os
import dotenv

dotenv.load_dotenv()
openai_token = os.getenv("OPENAI_API_KEY")


## Cargar y procesar documentos.
aqui cargamos los documentos sobre los que vamosa  aplicar el Q&A

Los dividimos en trozos (chunks). ASi buscamos los más relevantes y los pasamos al LLM.

In [3]:
# Load and process the text
loader = TextLoader('state_of_the_union.txt')
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

## Inicializamos PeristedChromaDB

Creamos los embeddings. Y los guardamos en la db

In [4]:

# Directorio donde se guardará la db
persist_directory = 'db'

engine = "gpt-4"
embedding = OpenAIEmbeddings(api_key=openai_token, model="text-embedding-3-large")

vectordb = Chroma.from_documents(documents=texts, embedding=embedding, persist_directory=persist_directory)

## Persist the Database
In a notebook, we should call `persist()` to ensure the embeddings are written to disk.
This isn't necessary in a script - the database will be automatically persisted when the client object is destroyed.

In [5]:
vectordb.persist()
vectordb = None

/opt/anaconda3/envs/devenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


## Cargamos la db del disco
`persist_directory` es el directorio donde se va a guardar la db y `embedding_function` tiene que ser la misma que cunado instanciamos a db. Se encargara de las respuestas.

In [8]:
# Cargamos la base de datos persistente del disco y usamos un langchain retrieval
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)
# Deprecated since version 0.1.17: Use create_retrieval_chain instead.
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(api_key=openai_token),
    retriever=vectordb.as_retriever(
        search_kwargs={"k": 4})
    )

In [6]:
# Cargamos la base de datos persistente del disco y usamos un langchain retrieval
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)
retriever = vectordb.as_retriever(search_kwargs={"k": 4})
llm=OpenAI(api_key=openai_token)
system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)
query = "What was the growth of manufacturing jobs in US last year"
chain.invoke({"input": query})



{'input': 'What was the growth of manufacturing jobs in US last year',
 'context': [Document(page_content='So let’s not wait any longer. Send it to my desk. I’ll sign it.  \n\nAnd we will really take off. \n\nAnd Intel is not alone. \n\nThere’s something happening in America. \n\nJust look around and you’ll see an amazing story. \n\nThe rebirth of the pride that comes from stamping products “Made In America.” The revitalization of American manufacturing.   \n\nCompanies are choosing to build new factories here, when just a few years ago, they would have built them overseas. \n\nThat’s what is happening. Ford is investing $11 billion to build electric vehicles, creating 11,000 jobs across the country. \n\nGM is making the largest investment in its history—$7 billion to build electric vehicles, creating 4,000 jobs in Michigan. \n\nAll told, we created 369,000 new manufacturing jobs in America just last year. \n\nPowered by people I’ve met like JoJo Burgess, from generations of union stee

In [12]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

prompthub = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever , "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)




rag_chain.invoke("What was the growth of manufacturing jobs in US last year")

'?\nSystem: 369,000 new manufacturing jobs were created in America just last year.'

## Preguntas y respuestas

Con langchainy RAG

In [37]:
query = "What was the growth of manufacturing jobs in US last year"
result = qa.invoke(query)
result.get("result")

' 369,000 new manufacturing jobs were created in America last year.'

## Limpieza

Cuando hayas terminado con la base de datos, puedes eliminarla del disco. Puedes eliminar la colección específica con la que está trabajando (si tiene varias) o eliminar toda la base de datos destruyendo el directorio de persistencia.

In [38]:
# limpiamos la colección
vectordb.delete_collection()
vectordb.persist()

# nos cargamos el directorio de persistencia
!rm -rf db/